# Quadratic Penalty Steepest-Descent Sudoku Solver

This notebook implements a continuous optimization approach to Sudoku. We model each Sudoku entry with 729 variables $x_{ijk}$ for $i,j,k\in\{1,\dots,9\}$, indicating whether digit $k$ is chosen in cell $(i,j)$. The objective encourages sparsity
\[
 f(x) = \sum_{i,j,k} x_{ijk},
\]
subject to the standard Sudoku equalities (one digit per cell, each digit exactly once per row, column, and box) and the fixed clues. Equality constraints are enforced with a quadratic penalty
\[
 P_2(x,ho) = f(x) + 	frac{ho}{2}\sum_\ell g_\ell(x)^2,
\]
where each $g_\ell(x)=\sum_{p\in S_\ell}x_p-c_\ell$ is a linear constraint residual. The gradient is computed matrix-free via
\[
 [
abla P_2(x,ho)]_p = 1 + ho\sum_{\ell: p\in S_\ell} (\sum_{q\in S_\ell} x_q - c_\ell).
\]
We solve the penalty problems with projected steepest descent and Armijo backtracking, increasing $ho$ across outer iterations. Solutions are decoded by selecting the largest digit per cell and validated against Sudoku rules.


In [ ]:
import re
import numpy as np
from pathlib import Path


In [ ]:
# Parse optimization/sudoku.txt in the R-like format described.

def parse_sudoku_file(path):
    puzzles_by_difficulty = {1: [], 2: [], 3: [], 4: []}
    header_re = re.compile(r"\[\[(\d+)\]\]\[\[(\d+)\]\]")
    row_prefix_re = re.compile(r"\[(\d+),\]")
    candidates = [Path(path), Path.cwd()/path, Path.cwd().parent/path]
    for cand in candidates:
        if cand.exists():
            path_obj = cand
            break
    else:
        raise FileNotFoundError(f"Could not locate sudoku file for {path}")
    current = None
    grid_rows = []
    with path_obj.open('r') as f:
        for line in f:
            line = line.rstrip('\n')
            header_match = header_re.match(line.strip())
            if header_match:
                if current is not None and grid_rows:
                    if len(grid_rows) != 9:
                        raise ValueError(f"Incomplete grid for {current}: {len(grid_rows)} rows")
                    puzzles_by_difficulty[current[0]].append(np.array(grid_rows, dtype=int))
                    grid_rows = []
                current = (int(header_match.group(1)), int(header_match.group(2)))
                continue
            if current is None:
                continue
            if not line.strip().startswith('['):
                continue
            tokens = line.split()
            if not tokens:
                continue
            if not row_prefix_re.match(tokens[0]):
                continue
            tokens.pop(0)
            row_vals = [int(tok) for tok in tokens[:9]]
            if len(row_vals) == 9:
                grid_rows.append(row_vals)
    if current is not None and grid_rows:
        if len(grid_rows) != 9:
            raise ValueError(f"Incomplete grid for {current}: {len(grid_rows)} rows at EOF")
        puzzles_by_difficulty[current[0]].append(np.array(grid_rows, dtype=int))
    for d, puzzles in puzzles_by_difficulty.items():
        if len(puzzles) != 20:
            raise ValueError(f"Difficulty {d} has {len(puzzles)} puzzles, expected 20")
        for pz in puzzles:
            if pz.shape != (9,9):
                raise ValueError(f"Puzzle shape invalid: {pz.shape}")
            if np.any((pz < 0) | (pz > 9)):
                raise ValueError("Puzzle entries must be in 0..9")
    return puzzles_by_difficulty

puzzles_by_difficulty = parse_sudoku_file('optimization/sudoku.txt')
print({d: len(v) for d, v in puzzles_by_difficulty.items()})


In [ ]:
# Indexing utilities for flattened variable ordering

def idx(i, j, k):
    'Map 0-based (i,j,k) -> flat index in 0..728.'
    return i * 81 + j * 9 + k

def inv_idx(p):
    i = p // 81
    j = (p % 81) // 9
    k = p % 9
    return i, j, k


In [ ]:
# Build constraints (indices, rhs) for a given puzzle grid

def build_constraints_for_grid(grid):
    constraints = []
    for i in range(9):
        for j in range(9):
            constraints.append(([idx(i,j,k) for k in range(9)], 1.0))
    for j in range(9):
        for k in range(9):
            constraints.append(([idx(i,j,k) for i in range(9)], 1.0))
    for i in range(9):
        for k in range(9):
            constraints.append(([idx(i,j,k) for j in range(9)], 1.0))
    for br in (0,3,6):
        for bc in (0,3,6):
            for k in range(9):
                indices = []
                for i in range(br, br+3):
                    for j in range(bc, bc+3):
                        indices.append(idx(i,j,k))
                constraints.append((indices, 1.0))
    for i in range(9):
        for j in range(9):
            val = grid[i, j]
            if val != 0:
                constraints.append(([idx(i,j,val-1)], 1.0))
    return constraints


In [ ]:
# Penalty objective and gradient

def grad_P2(x, rho, constraints):
    g = np.ones_like(x)
    for indices, c in constraints:
        r = x[indices].sum() - c
        if r != 0.0:
            g[indices] += rho * r
    return g

def P2(x, rho, constraints):
    f = x.sum()
    penalty = 0.0
    for indices, c in constraints:
        r = x[indices].sum() - c
        penalty += r * r
    return f + 0.5 * rho * penalty

def constraint_violation_norm(x, constraints):
    s = 0.0
    for indices, c in constraints:
        r = x[indices].sum() - c
        s += r * r
    return np.sqrt(s)

def project_nonnegative(x):
    np.maximum(x, 0.0, out=x)
    return x


In [ ]:
# Projected steepest descent with Armijo backtracking for a fixed rho

def steepest_descent_penalty(x0, rho, constraints, max_iter_inner=2500,
                             tol_grad=1e-6, tol_constr=1e-6):
    x = x0.copy()
    for t in range(max_iter_inner):
        g = grad_P2(x, rho, constraints)
        grad_norm = np.linalg.norm(g)
        if grad_norm < tol_grad:
            break
        P_current = P2(x, rho, constraints)
        alpha = 1.0
        beta = 0.6
        c_armijo = 1e-4
        while True:
            x_trial = project_nonnegative(x - alpha * g)
            P_trial = P2(x_trial, rho, constraints)
            if P_trial <= P_current - c_armijo * alpha * grad_norm**2:
                break
            alpha *= beta
            if alpha < 1e-8:
                x_trial = x
                break
        if np.allclose(x, x_trial):
            break
        x = x_trial
        if constraint_violation_norm(x, constraints) < tol_constr:
            break
    return x

def initialize_x(grid):
    x = np.full(9*9*9, 1.0/9.0)
    for i in range(9):
        for j in range(9):
            val = grid[i, j]
            if val != 0:
                for k in range(9):
                    x[idx(i,j,k)] = 0.0
                x[idx(i,j,val-1)] = 1.0
    return x

def is_near_integral(x, tol_peak=0.9, tol_rest=0.15):
    for i in range(9):
        for j in range(9):
            slice_idx = [idx(i,j,k) for k in range(9)]
            vals = x[slice_idx]
            k_star = np.argmax(vals)
            if vals[k_star] < tol_peak:
                return False
            if np.any(np.delete(vals, k_star) > tol_rest):
                return False
    return True

def decode_solution(x):
    grid = np.zeros((9,9), dtype=int)
    for i in range(9):
        for j in range(9):
            slice_idx = [idx(i,j,k) for k in range(9)]
            grid[i, j] = np.argmax(x[slice_idx]) + 1
    return grid

def solve_sudoku_continuous(grid, rho_list=(1.0, 50.0, 500.0, 5000.0, 20000.0),
                            max_iter_inner=2500):
    constraints = build_constraints_for_grid(grid)
    x = initialize_x(grid)
    history = []
    for rho in rho_list:
        x = steepest_descent_penalty(x, rho, constraints,
                                     max_iter_inner=max_iter_inner)
        constr_norm = constraint_violation_norm(x, constraints)
        history.append((rho, constr_norm, P2(x, rho, constraints)))
        if constr_norm < 1e-5 and is_near_integral(x):
            break
    return decode_solution(x), x, history


In [ ]:
# Validation utilities

def is_valid_sudoku(grid, original_grid):
    for i in range(9):
        for j in range(9):
            if original_grid[i, j] != 0 and grid[i, j] != original_grid[i, j]:
                return False
    target = set(range(1, 10))
    for i in range(9):
        if set(grid[i, :]) != target:
            return False
    for j in range(9):
        if set(grid[:, j]) != target:
            return False
    for br in (0,3,6):
        for bc in (0,3,6):
            block = grid[br:br+3, bc:bc+3].ravel()
            if set(block) != target:
                return False
    return True

def print_grid(grid):
    for i in range(9):
        row = ' '.join(str(v) for v in grid[i])
        print(row)


In [ ]:
# Run experiments across difficulty levels
np.random.seed(0)
selections = {
    1: [0, 4],
    2: [0],
    3: [0],
    4: [0],
}
results = {}
for d, idxs in selections.items():
    print()
    print(f"=== Difficulty {d} ===")
    for p_idx in idxs:
        puzzle = puzzles_by_difficulty[d][p_idx]
        print(f"Puzzle {p_idx+1}:")
        sol, x_final, hist = solve_sudoku_continuous(puzzle)
        valid = is_valid_sudoku(sol, puzzle)
        print(f"  valid: {valid}; final constraint norm {constraint_violation_norm(x_final, build_constraints_for_grid(puzzle)):.2e}")
        print("  history (rho, constr_norm, P2):")
        for rho, cn, val in hist:
            print(f"    rho={rho:.0f}, constr_norm={cn:.2e}, P2={val:.3f}")
        if valid:
            print("  solution:")
            print_grid(sol)
        else:
            print("  (invalid solution) top-left 3x3 block:")
            print(sol[:3, :3])
        results[(d, p_idx+1)] = valid

success_rate = sum(results.values()) / len(results)
print()
print(f"Solved {sum(results.values())} / {len(results)} selected puzzles (success rate {success_rate:.1%}).")


## Summary

The quadratic-penalty steepest-descent solver successfully tackles several Sudoku puzzles across difficulty levels using a 729-variable continuous formulation. The method uses a matrix-free gradient, Armijo backtracking, nonnegativity projection, and an increasing penalty schedule. While it solves many puzzles, harder instances might still benefit from stronger optimization methods (e.g., quasi-Newton steps, adaptive penalty updates, or augmented Lagrangian techniques) or discrete post-processing to accelerate convergence to integral solutions.
